**Install All the Required Packages**

In [ ]:
!pip install ultralytics

**Import All the Required Libraries**

In [ ]:
import ultralytics
from ultralytics import YOLO
from IPython.display import Image

**Check Ultralytics Version**

In [ ]:
ultralytics.checks()

**Download the Dataset from Roboflow**

In [ ]:
!pip install roboflow

**Clear out old directories**

In [ ]:
! rm -rf plants-classification-3/.ipynb_checkpoints/

In [ ]:
! rm -rf runs/

**Download the dataset from Roboflow**

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="z1uD0b9XOZdnivUdFkEd")
project = rf.workspace("rockpaperscissors-adtav").project("plants-classification-zcckg")
version = project.version(3)
dataset = version.download("folder")
# "error": "yolov11 is an invalid format for project type classification. Please use one of: folder, clip."
# dataset = version.download("yolov11") # not a valid format

# NOTE: You must rename the 'valid' folder to 'val' in order for YOLO to work.

**Load the YOLO11 Model**

In [ ]:
model = YOLO("yolo11s-cls.pt") # Classification

**Train the YOLO11 Classification Model**

In [ ]:
data_path = "/content/plants-classification-3"
results = model.train (task = "classify", mode = "train", data = data_path, epochs = 5)

**Results Analysis**

**Confusion Matrix**

In [ ]:
Image("/content/runs/classify/train/confusion_matrix.png", width = 800)

**Normalized Confusion Matrix**

In [ ]:
Image("/content/runs/classify/train/confusion_matrix_normalized.png", width= 800)

**Model Prediction on Validation Batch**

In [ ]:
Image("/content/runs/classify/train/val_batch0_pred.jpg", width=800)

In [ ]:
Image("/content/runs/classify/train/val_batch1_pred.jpg", width=800)

**Training and Validation Loss**

In [ ]:
Image("/content/runs/classify/train/results.png", width = 800)

**Predict on Test Data using Trained Model**

In [ ]:
# Download the Model Weights from Drive
!gdown "https://drive.google.com/uc?id=1vQ4W4nksiq6yKyNiUkVUGQ7Bb-SxYJSB&confirm=t"

In [ ]:
# Load the Trained Classification Model
model = YOLO("/content/best.pt")

In [ ]:
# Re-organize the test set image files

# from pathlib import Path
# # Define the root directory to start searching from
# root_folder = "/content/plants-classification-3/test"

# # Use rglob() with pattern matching for both cases
# # The list comprehension makes the search case-insensitive for common jpg extensions
# images_files = [str(file_path) for file_path in Path(root_folder).rglob('*.[jJ][pP][gG]')]

# # Print the list of found files
# for file_path in jpg_files:
#     print(file_path)

# test_folder = "/content/plants-classification-3/test2"

In [ ]:
test_folder = "/content/plants-classification-3/test"

In [ ]:
# Loop through each class folder in the test directory
import os
correct = 0
incorrect = 0
for class_folder in os.listdir(test_folder):
  class_folder_path = os.path.join(test_folder, class_folder)
  # print(class_folder_path)
  # Make sure it's a directory
  if os.path.isdir(class_folder_path):
    ground_truth_class = class_folder # This is the ground truth class based on the folder name
    print(f"Processing Class Folder: {ground_truth_class}\n")
    # List All Images in the Current Class Folder
    image_files = [os.path.join(class_folder_path, img) for img in os.listdir(class_folder_path) if img.endswith(('.jpg', '.jpeg', '.png'))]
    # Predict on each image in the class folder
    for img_path in image_files:
      # Perform Classification on the Image
      results = model.predict (img_path, conf = 0.25) # , verbose=False)
      # Loop through the results and extract predictions
      for result in results:
        # Get the predicted class index (top-1) and confidence
        predicted_class_index = result.probs.top1 # Get the predicted class index (top-1)
        predicted_class = result.names[predicted_class_index] # Map index to class name
        confidence = result.probs.top1conf # Get the confidence score for the top prediction
        # Update the accuracy calculation
        if predicted_class == ground_truth_class:
          correct += 1
        else:
          incorrect += 1
        # Display the prediction results for the image along with the ground truths
        print(f"Image: {img_path}")
        print(f"Ground Truth: {ground_truth_class}, Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")

print(f"Correct = {correct}, Incorrect = {incorrect}, Accuracy = {correct / (correct + incorrect)}")
